# Felipe Cardona - Tarea 3 Clasificación De Datos Utilizando Imágenes

## Configurar e Importar dependencias necesarias.

In [5]:
pip install -r requirements.txt

  Using cached tensorflow-2.17.1-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached scikit_learn-1.5.2-cp312-cp312-win_amd64.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 4.2 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.0 MB 11.7 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 22.4 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 20.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/382.4 MB ? eta -:--:--
   - -------------------------------------- 13.4/382.4 MB 70.1 MB/s eta 0:00:06
   -- ------------------------------------- 24.6/382.4 MB 60.0 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 0:00:06
   -- ------------------------------------- 28.6/382.4 MB 60.5 MB/s eta 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [33]:
import os
import glob
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

## Cargar las imagenes para entrenamiento

Al momento de cargar nos encargaremos de eliminar todas aquellas imagenes corruptas que no sean posible de leer y tambien de normalizar las imagenes.

In [49]:
trainPath = 'data/train'
# Defino las dimensiones de las imágenes para estandarizar el procesamiento
IMG_SIZE_LR = 64  # Tamaño reducido para el modelo de regresión logística
IMG_SIZE_CNN = 128      # Tamaño para el modelo CNN

# Mapeo de etiquetas para corrección de nombres
labelMapping = {
    "Shepherds Purse": "Shepherd’s Purse",
}

def loadImages(path, imgSize=IMG_SIZE_LR, grayscale=False):
    """
    Carga y procesa las imágenes desde el directorio especificado.
    
    Parámetros:
    - path (str): Ruta de la carpeta de imágenes.
    - imgSize (int): Tamaño al que se redimensionarán las imágenes.
    - grayscale (bool): Si es True, carga las imágenes en escala de grises. Por defecto es False (color).
    
    Retorna:
    - images (np.array): Arreglo de imágenes procesadas.
    - labels (np.array): Arreglo de etiquetas correspondientes.
    """
    images = []
    labels = []
    for folder in os.listdir(path):
        label = labelMapping.get(folder, folder)
        folderPath = os.path.join(path, folder)
        if not os.path.isdir(folderPath):
            continue
        for file in glob.glob(os.path.join(folderPath, "*.png")):
            # Cargar la imagen en escala de grises si está especificado
            if grayscale:
                img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            else:
                img = cv2.imread(file)
            if img is None:
                print(f"Error al cargar la imagen: {file}")
                continue
            img = cv2.resize(img, (imgSize, imgSize))
            img = img / 255.0  # Normalización
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

### Pasaremos a codificar etiquetas y dividir los datos los datos en entrenamiento y validación

In [53]:
X, y = loadImages(trainPath,IMG_SIZE_LR,True) # Cargar imágenes y etiquetas para el modelo de regresión logística

# Codificar las etiquetas
labelEncoder = LabelEncoder()
yEncoded = labelEncoder.fit_transform(y)
yEncoded = to_categorical(yEncoded)

# Dividir en conjuntos de entrenamiento y validación
XTrain, XVal, yTrain, yVal = train_test_split(X, yEncoded, test_size=0.2, random_state=42)

### Pasaremos a realizar ajustes y a entrenar el modelo de Regresión Logistica

In [54]:
# Aplanar las imágenes
XTrainFlat = XTrain.reshape(XTrain.shape[0], -1)
XValFlat = XVal.reshape(XVal.shape[0], -1)

In [89]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Crear el modelo de regresión logística con un pipeline
logisticModel = make_pipeline(
    LogisticRegression(max_iter=1500, solver='lbfgs', C=1)
)

logisticModel.fit(XTrainFlat, yTrain.argmax(axis=1))

Pipeline(steps=[('logisticregression', LogisticRegression(C=1, max_iter=1500))])

### Realizando predicciones con los datos de entrenamiento conseguimos los siguientes resultados

In [90]:
# Evaluar el modelo de Regresión Logística
yValPred = logisticModel.predict(XValFlat)
accuracy = accuracy_score(yVal.argmax(axis=1), yValPred)
confMatrix = confusion_matrix(yVal.argmax(axis=1), yValPred)

print("Precisión del modelo de Regresión Logística:", accuracy)
print("Matriz de Confusión:\n", confMatrix)

Precisión del modelo de Regresión Logística: 0.13123844731977818
Matriz de Confusión:
 [[ 4  4  2  8  0  9 13  1  5  0  9  1  0  0  0  3]
 [ 0  8  2 16  3  5 10  1 16  4 17  5  0  0  0  0]
 [ 0  6  2  8  2  3  9  0 12  6  8  1  0  0  0  0]
 [ 3  6  3 34  1 12 17  2 18  3 11  7  1  0  2  0]
 [ 1  3  0  4  0  9 16  1  6  2  8  5  1  0  0  0]
 [ 3  4  5 17  1 17 20  4 12  3  7  6  0  0  0  0]
 [ 5 12  5 17  1 19 20  2 17  8 10  5  0  0  1  0]
 [ 0  1  2 11  0  5 10  0  3  2  6  0  0  1  0  0]
 [ 0  4  6 28  1 12 19  5 19  4  9  6  0  0  0  0]
 [ 0  2  2  3  3  2  9  1 10  1  9  1  0  0  0  0]
 [ 3  7  8 12  1  5 13  7 16  2 14  0  0  0  0  0]
 [ 4  5  2 13  1  7 17  2  7  3  8  8  0  0  0  2]
 [ 1  0  2  1  1  3  1  0  1  1  0  1  7  2  8  6]
 [ 2  0  2  1  1  5  2  2  1  0  0  1  2  3  2  2]
 [ 2  0  2  6  0  2  3  0  1  0  0  2  4  1  3  4]
 [ 1  1  5  1  0  5  2  1  1  1  1  1  2  1  2  2]]


### Ahora cargaremos las imagenes de test con el fin de poner a prueba la predicción del modelo.

In [63]:
testPath = "data/test"

def loadTestImages(path, imgSize=IMG_SIZE_LR, grayscale=False):
    """
    Carga y procesa las imágenes de prueba desde el directorio especificado.
    
    Parámetros:
    - path (str): Ruta de la carpeta de imágenes de prueba.
    - imgSize (int): Tamaño al que se redimensionarán las imágenes.
    - grayscale (bool): Si es True, carga las imágenes en escala de grises. Por defecto es False (color).
    
    Retorna:
    - testImages (np.array): Arreglo de imágenes de prueba procesadas.
    - filenames (list): Lista de nombres de archivo correspondientes.
    """
    testImages = []
    filenames = []
    filePaths = glob.glob(os.path.join(path, "*.png"))
    for file in filePaths:
        # Cargar la imagen en escala de grises si está especificado
        if grayscale:
            img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        else:
            img = cv2.imread(file)
        if img is None:
            print(f"Error al cargar la imagen: {file}")
            continue
        img = cv2.resize(img, (imgSize, imgSize))
        img = img / 255.0  # Normalización
        testImages.append(img)
        filenames.append(os.path.basename(file))
    return np.array(testImages), filenames

### Creamos la función que realice la predicción y genere el archivo para subir a Kaggle.

In [69]:
# Función para realizar predicciones y guardar el archivo CSV para Kaggle
import pandas as pd

def predictAndSave(model, xTest, filenames, labelEncoder, outputFile="submission.csv"):
    """
    Realiza predicciones usando el modelo dado y guarda los resultados en un archivo CSV.
    
    Parámetros:
    - model: Modelo entrenado para realizar predicciones.
    - xTest (np.array): Imágenes de prueba en formato aplanado.
    - filenames (list): Lista de nombres de archivo correspondientes a cada imagen de prueba.
    - labelEncoder (LabelEncoder): Codificador de etiquetas para decodificar las predicciones.
    - outputFile (str): Nombre del archivo de salida CSV.
    """
    predictions = model.predict(xTest)
    # Convertir predicciones a etiquetas decodificadas
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictedLabels = labelEncoder.inverse_transform(np.argmax(predictions, axis=1))
    else:
        predictedLabels = labelEncoder.inverse_transform(predictions)
    # Crear un DataFrame con las predicciones
    df = pd.DataFrame({"file": filenames, "label": predictedLabels})
    # Guardar el archivo CSV
    df.to_csv(outputFile, index=False)
    print(f"Archivo de envío guardado como: {outputFile}")

### Realizando predicción del modelo de Regresión Logistica

In [91]:
# Aplanar imágenes de prueba para el modelo de regresión logística
Xtest, testFilenames = loadTestImages(testPath, imgSize=IMG_SIZE_LR, grayscale=True)
XtestFlat = Xtest.reshape(Xtest.shape[0], -1)

predictAndSave(logisticModel, XtestFlat, testFilenames, labelEncoder, outputFile="logistic_regression.csv")

Archivo de envío guardado como: logistic_regression.csv


## Configuración de la Red Convolucional (CNN)
### Primero importamos las librerias necesarias

In [122]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

### Construyendo el Modelo

In [123]:
inputShape = (64, 64, 1)  # Si decides usar blanco y negro, ajusta a (64, 64, 1)
numClasses = yEncoded.shape[1]

# Construcción del modelo CNN
def buildCnnModel(inputShape, numClasses):
    model = Sequential()
    # Bloque de Convolución y Pooling 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=inputShape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    # Bloque de Convolución y Pooling 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    # Bloque de Convolución y Pooling 3
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    # Aplanamiento y Capa densa
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # Capa de salida
    model.add(Dense(numClasses, activation='softmax'))
    # Compilación del modelo
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnnModel = buildCnnModel(inputShape, numClasses)
cnnModel.summary()

C:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 29, 29, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │         589,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 16)                  │           2,064 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 686,096 (2.62 MB)

 Trainable params: 685,392 (2.61 MB)

 Non-trainable params: 704 (2.75 KB)

### Habiendo creado el modelo ahora pasaremos a entrenarlo

In [124]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Entrenamiento del modelo CNN
history = cnnModel.fit(
    XTrain, yTrain,
    validation_data=(XVal, yVal),
    epochs=50,  # Puedes ajustar según el rendimiento del modelo
    batch_size=32,
    callbacks=[earlyStopping, reduceLr]
)

Epoch 1/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 24s 152ms/step - accuracy: 0.1490 - loss: 3.1292 - val_accuracy: 0.0813 - val_loss: 3.3132 - learning_rate: 0.0010
Epoch 2/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.2561 - loss: 2.3394 - val_accuracy: 0.0434 - val_loss: 4.9599 - learning_rate: 0.0010
Epoch 3/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 19s 138ms/step - accuracy: 0.3398 - loss: 2.0155 - val_accuracy: 0.0952 - val_loss: 4.9481 - learning_rate: 0.0010
Epoch 4/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 20s 145ms/step - accuracy: 0.4067 - loss: 1.8132 - val_accuracy: 0.2015 - val_loss: 3.2934 - learning_rate: 0.0010
Epoch 5/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 20s 144ms/step - accuracy: 0.4567 - loss: 1.6116 - val_accuracy: 0.3512 - val_loss: 2.0200 - learning_rate: 0.0010
Epoch 6/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 19s 143ms/step - accuracy: 0.5417 - loss: 1.3498 - val_accuracy: 0.5120 - val_loss: 1.5020 - learning_rate: 0.0010
Epoch 7/50
136/136 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.5

### Evaluando el modelo

In [125]:
valLoss, valAccuracy = cnnModel.evaluate(XVal, yVal)
print(f"Precisión en el conjunto de validación: {valAccuracy}")

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7217 - loss: 0.9389
Precisión en el conjunto de validación: 0.7144177556037903


### Probando el modelo

In [126]:
# Cargar datos de test nuevamente para imágenes de prueba en blanco y negro
xTest, testFilenames = loadTestImages(testPath, IMG_SIZE_LR, grayscale=True)
# Ajustar el canal de entrada para la CNN
xTest = xTest.reshape(xTest.shape[0], IMG_SIZE_LR, IMG_SIZE_LR, 1)
# Realizar predicciones y guardarlas en CSV
print("Forma de xTest:", xTest.shape)
print("Forma de entrada de la CNN:", cnnModel.input_shape)
# predictAndSave(cnnModel, xTest, testFilenames, labelEncoder, outputFile="cnn_predictions.csv")

Forma de xTest: (962, 64, 64, 1)
Forma de entrada de la CNN: (None, 64, 64, 1)


In [127]:
# Imprime la forma de las entradas esperadas y de las entradas reales para verificar
test_sample_adjusted = test_sample.reshape(1, IMG_SIZE_LR, IMG_SIZE_LR, 1)
try:
    single_prediction = cnnModel.predict(test_sample_adjusted)
    print("Predicción para una sola imagen (ajustada):", single_prediction)
except ValueError as e:
    print("Error en la predicción con la muestra ajustada:", str(e))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Predicción para una sola imagen (ajustada): [[2.7826505e-05 4.1565538e-05 7.3605137e-05 2.4106132e-06 1.8653001e-06
  1.4249616e-05 1.3304256e-04 2.2715608e-06 1.6020220e-05 1.1767039e-04
  9.9948078e-01 6.5903441e-05 1.5882440e-05 1.0076185e-06 3.6028532e-06
  2.2450813e-06]]


In [128]:
predictAndSave(cnnModel, xTest, testFilenames, labelEncoder, outputFile="cnn_predictions.csv")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Archivo de envío guardado como: cnn_predictions.csv
